In [1]:
"""Take the Institution name as input. Use Pydantic to define the schema for the desired output and create a custom output parser.
Invoke the Chain and Fetch Results. Extract the below Institution related details from Wikipedia:
The founder of the Institution. When it was founded.
The current branches in the institution. How many employees are working in it. A brief 4-line summary of the institution."""


# Install required libraries
!pip install wikipedia-api pydantic ipywidgets

from pydantic import BaseModel
from typing import List, Optional
import wikipediaapi
from IPython.display import display
import ipywidgets as widgets

# Define the Pydantic model
class InstitutionDetails(BaseModel):
    founder: Optional[str]
    founded: Optional[str]
    branches: Optional[List[str]]
    number_of_employees: Optional[int]
    summary: Optional[str]

# Function to fetch institution details
def fetch_institution_details(institution_name: str) -> InstitutionDetails:
    # Set user agent as required by Wikipedia API
    user_agent = "MyJupyterNotebook/1.0 (contact: myemail@example.com)"
    wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='en')

    page = wiki_wiki.page(institution_name)

    if not page.exists():
        raise ValueError(f"The page for '{institution_name}' does not exist on Wikipedia.")

    # Extract summary
    summary = page.summary[:500]  # Truncate for display

    # Initialize extracted fields
    founder = None
    founded = None
    branches = []
    number_of_employees = None

    # Heuristic parsing of raw text (very rough)
    lines = page.text.split('\n')
    for line in lines:
        line_lower = line.lower()
        if 'founder' in line_lower:
            founder = line.split(':')[-1].strip()
        elif 'founded' in line_lower:
            founded = line.split(':')[-1].strip()
        elif 'branches' in line_lower:
            branches = [b.strip() for b in line.split(':')[-1].split(',')]
        elif 'number of employees' in line_lower:
            try:
                number_of_employees = int(line.split(':')[-1].replace(',', '').strip())
            except ValueError:
                number_of_employees = None

    return InstitutionDetails(
        founder=founder,
        founded=founded,
        branches=branches if branches else None,
        number_of_employees=number_of_employees,
        summary=summary
    )

# Display function
def display_institution_details(details: InstitutionDetails):
    print(f"🔹 Founder: {details.founder or 'N/A'}")
    print(f"🔹 Founded: {details.founded or 'N/A'}")
    print(f"🔹 Branches: {', '.join(details.branches) if details.branches else 'N/A'}")
    print(f"🔹 Number of Employees: {details.number_of_employees or 'N/A'}")
    print(f"\n📝 Summary:\n{details.summary or 'N/A'}")

# Button click event handler
def on_button_click(b):
    institution_name = text_box.value.strip()
    if not institution_name:
        print("Please enter a valid institution name.")
        return
    try:
        details = fetch_institution_details(institution_name)
        display_institution_details(details)
    except ValueError as e:
        print(e)

# Widgets: input + button
text_box = widgets.Text(
    value='',
    placeholder='Enter the institution name',
    description='Institution:',
    disabled=False
)

button = widgets.Button(
    description='Fetch Details',
    disabled=False,
    button_style='info',
    tooltip='Click to fetch institution details',
    icon='search'
)

button.on_click(on_button_click)

# Display the input and button
display(text_box, button)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=3d7bb881063593af37cfa6d1a370b7a6148065479433d750d0ab3aab879aec5d
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


Text(value='', description='Institution:', placeholder='Enter the institution name')

Button(button_style='info', description='Fetch Details', icon='search', style=ButtonStyle(), tooltip='Click to…

🔹 Founder: MIT faculty members have often been recruited to lead other colleges and universities. Founding faculty-member Charles W. Eliot became president of Harvard University in 1869, a post he would hold for 40 years, during which he wielded considerable influence both on American higher education and on secondary education. MIT alumnus and faculty member George Ellery Hale played a central role in the development of the California Institute of Technology (Caltech), and other faculty members have been key founders of Franklin W. Olin College of Engineering in nearby Needham, Massachusetts.
🔹 Founded: MIT alumni have led prominent institutions of higher education, including the University of California system, Harvard University, the New York Institute of Technology, Johns Hopkins University, Carnegie Mellon University, Tufts University, Rochester Institute of Technology, Rhode Island School of Design (RISD), UC Berkeley College of Environmental Design, the New Jersey Institute of T